In [1]:
# %%

import ants
import numpy as np
from utils.load_scalar import image_read_rotation, load_scalar
import pandas as pd
import os

# %%
if not os.path.exists('outputs/registration_labels'):
    os.makedirs('outputs/registration_labels')

if not os.path.exists('outputs/registration_images'):
    os.makedirs('outputs/registration_images')

if not os.path.exists('outputs/rotation_images'):
    os.makedirs('outputs/rotation_images')

if not os.path.exists('outputs/csf_segmentation'):
    os.makedirs('outputs/csf_segmentation')
    

# %%
clinical_df = pd.read_csv('data/summary/ONDRI_summary.csv')
clinical_df = clinical_df[clinical_df['COHORT']!= 'VCI']
atlas = ants.image_read('atlas/mni_icbm152_nlin_sym_09c_t2_csfnull.nii.gz')
atlas_labels = ants.image_read('atlas/mni_icbm152_nlin_sym_09c_CerebrA_nifti/mni_icbm152_CerebrA_tal_nlin_sym_09c.nii') 
atlas_brainmask = ants.image_read('atlas/mni_icbm152_nlin_sym_09c/mni_icbm152_t1_tal_nlin_sym_09c_mask.nii').numpy()





In [2]:
atlas_brain = atlas.copy()
atlas_brain[atlas_brainmask == 0] = 0

In [3]:
csf_atlas = ants.image_read('atlas/mni_icbm152_nlin_sym_09c/mni_icbm152_csf_tal_nlin_sym_09c.nii')
wm_atlas = ants.image_read('atlas/mni_icbm152_nlin_sym_09c/mni_icbm152_wm_tal_nlin_sym_09c.nii')
gm_atlas = ants.image_read('atlas/mni_icbm152_nlin_sym_09c/mni_icbm152_gm_tal_nlin_sym_09c.nii')

In [6]:
mask_loc = []
# %%
for i, row in clinical_df.iterrows():
    print('Processing %s/%s: %s' % (str(i), str(len(clinical_df)), row['STANDARDIZED_NII']))
    output_picture = 'outputs/registration_images/%s'% row['STANDARDIZED_NII'].replace('.nii.gz', '.png')
    if os.path.exists(output_picture):
        print('Already completed %s' % output_picture)
        continue
    
    img_path = 'data/FLAIR_standardized/%s' % row['STANDARDIZED_NII']
    
    img0 = image_read_rotation(img_path, k=1)

    if os.path.exists('data/mask/%s' % row['MASK_NII']):
        img_mask = image_read_rotation('data/mask/%s' % row['MASK_NII'], k=1)
        mask_loc.append('data/mask/%s' % row['MASK_NII'])
    else: 
        img_mask = image_read_rotation('data/hdbet_mask/%s' % row['STANDARDIZED_NII'].replace('.nii.gz', '_mask.nii.gz'), k=1)
        mask_loc.append('data/mask/%s' % row['STANDARDIZED_NII'].replace('.nii.gz', '_mask.nii.gz'))


    if not img_mask.shape == img0.shape:
        img_mask = ants.pad_image(ants.from_numpy(img_mask), shape=img0.shape).numpy()
    img_brain = img0.copy()
    img_brain[img_mask == 0] = 0

    try:
        registration = ants.registration(fixed=atlas_brain, moving=img_brain, type_of_transform='SyNAggro')
        registered_labels = ants.apply_transforms(fixed=img0, moving=atlas_labels, transformlist=registration['invtransforms'], interpolator='genericLabel')

        registered_labels[img_mask == 0] = 0
        registered_labels.to_file('outputs/registration_labels/%s'% row['STANDARDIZED_NII'])
        img0.to_file('outputs/rotation_images/%s'% row['STANDARDIZED_NII'])

        img0.plot(overlay=registered_labels, overlay_cmap='tab20', nslices=6, overlay_alpha=0.5, axis=2, figsize=3, filename='outputs/registration_images/%s'% row['STANDARDIZED_NII'].replace('.nii.gz', '.png'))

        csf_segmentation =  ants.apply_transforms(fixed=img0, moving=csf_atlas, transformlist=registration['invtransforms'])
        parenchyma_segmentation =  ants.apply_transforms(fixed=img0, moving=wm_atlas+gm_atlas, transformlist=registration['invtransforms'])
        seg = ants.prior_based_segmentation(img0,[csf_segmentation, parenchyma_segmentation], img_mask!= 0, priorweight=0.25, mrf=0.1, iterations=25)
        seg['segmentation'].to_filename('outputs/csf_segmentation/%s'% row['STANDARDIZED_NII'])
    except RuntimeError:
        print('Error in registration for %s' % row['STANDARDIZED_NII'])
        continue



Processing 1/375: sub-OND01BYC1008_ses-01SE01MR_run-1_FLAIR_time0.nii.gz
